In [226]:
import pandas as pd 
pd.options.mode.chained_assignment = None
import numpy as np
from astropy.table import Table
import matplotlib.pyplot as plt
import matplotlib
from multiprocessing import Pool
# from profilestats import profile
from scipy.stats import binned_statistic
from funcs.analysis import *
%matplotlib inline

def reader(n_subarray):
    return pd.read_csv('data/merged/{}/lc_{}_{}.csv'.format(obj,band,n_subarray), nrows=None, index_col = ID, dtype = {'catalogue': np.uint8, 'mag_ps': np.float32, 'magerr': np.float32, 'mjd': np.float64, ID: np.uint32})

In [227]:
obj = 'qsos'
ID  = 'uid'
band = 'r'
dr = analysis(band, ID)

In [228]:
dr.read_in(reader, redshift=True)
dr.group(keys = ['uid'],read_in=True)

/disk1/hrb/anaconda3/envs/astroconda/lib/python3.6/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
/disk1/hrb/anaconda3/envs/astroconda/lib/python3.6/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
/disk1/hrb/anaconda3/envs/astroconda/lib/python3.6/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
/disk1/hrb/anaconda3/envs/astroconda/lib/python3.6/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [211]:
pgq = pd.read_csv('data/catalogues/PGQ.txt', delimiter='\s+', skiprows=1, names=['name','ra','dec','phot_bmag','comments','pg_class','Search_Offset'])
pgq['pgq_id'] = np.arange(1,125)
pgq = pgq.set_index('pgq_id')

In [212]:
pgq

,name,ra,dec,phot_bmag,comments,pg_class,Search_Offset
pgq_id,,,,,,,
1,PG 0003+159,1.496833,16.163611,15.96,PHL 658,QSO,973.855778
2,PG 0003+199,1.581417,20.202944,13.75,MK 335,Sey,1215.729887
3,PG 0007+107,2.629208,10.974861,16.11,III ZW2,QSO,676.898795
4,PG 0026+129,7.307083,13.267778,14.95,0.142,QSO,906.910602
5,PG 0043+039,11.446750,4.173139,15.88,0.384,QSO,730.451086
...,...,...,...,...,...,...,...
120,PG 2302+029,346.187292,3.196083,16.03,1.044,QSO,850.238652
121,PG 2304+043,346.762167,4.549194,15.44,0.042,Sey,839.068812
122,PG 2308+099,347.823958,10.137722,16.12,4C 09.32,QSO,947.685489


In [224]:
def search_pgq(self, pgq, threshold_arcsec):
    """
    Search database by ra and dec and return objects within threshold 

    Parameters
    ----------
    ra_dec : numpy array
        list of ra and dec, with shape (N,2)
    threshold : float
        measured in arcseconds
    Returns
    -------
    found : pandas.DataFrame of uid, ra, dec, distance of object to given ra, dec
        desc
    """
    ra_dec = pgq[['ra','dec']].values

    ra1 = self.coords['ra'].values
    dec1= self.coords['dec'].values
    ra2 = ra_dec[:,0,np.newaxis]
    dec2= ra_dec[:,1,np.newaxis]

    ra_sep = (ra1-ra2)*np.cos( np.deg2rad( 0.5*(dec1+dec2) ) )
    dec_sep= (dec1-dec2)
    dist = (ra_sep**2 + dec_sep**2)**0.5 * 3600
    boolean = (dist<threshold_arcsec)
    offset  = pd.Series(dist[boolean], name='dist')
    found   = self.coords[boolean.any(axis=0)]
    lookup = pd.DataFrame({'pgq_id':pgq[boolean.any(axis=1)].index,'uid':found.index}).set_index('pgq_id')
    found['dist_arcsec'] = offset.values
    print('Number of matched quasars within {}": {}'.format(threshold_arcsec, len(found.index)))
    return found, lookup

In [225]:
found, lookup = search_pgq(dr, pgq, threshold_arcsec=2)

Number of matched quasars within 2": 59


In [215]:
pgq_matched = pgq.join(lookup.astype({'uid':'int'}), on='pgq_id', how='left')
pgq_matched = pgq_matched.dropna(subset=['uid']).astype({'uid':'uint32'})

In [216]:
pgq_matched

,name,ra,dec,phot_bmag,comments,pg_class,Search_Offset,uid
pgq_id,,,,,,,,
12,PG 0157+002,29.959375,0.394667,15.20,MK 1014,QSO,1797.703986,76294
14,PG 0832+252,128.899208,24.994500,15.90,0.331,QSO,7481.431408,132987
17,PG 0906+484,137.541708,48.228278,16.46,NaN,QSO,7166.256288,159016
20,PG 0923+201,141.478000,19.901417,16.04,0.190,QSO,8241.708628,171762
21,PG 0931+437,143.760583,43.519639,16.41,0.457 U,QSO,7547.599284,178761
23,PG 0935+417,144.737583,41.472556,16.30,1.937 K,QSO,7663.159013,181670
24,PG 0936+396,144.935583,39.400806,16.30,0.458 K,QSO,7754.001044,182225
25,PG 0946+302,147.421292,29.922028,16.00,1.216,QSO,8214.850071,189762
26,PG 0947+397,147.701625,39.447361,16.40,0.206 K,QSO,7844.865853,190644


In [217]:
pgq_matched['uid'].values[:1]

array([76294], dtype=uint32)

In [218]:
output_notebook()

Loading BokehJS ...

In [219]:
dr.summary()

Number of qsos with lightcurves in r band : 524,278
Number of datapoints in:
SDSS: 1,433,284
PS: 6,222,972
ZTF: 28,632,287


In [220]:
uids = pgq_matched['uid'][pgq_matched['uid'].isin(dr.idx_uid)]
len(uids)

58

> photometry of the 58 matched PG quasars

In [223]:
dr.plot_series_bokeh(uids)